In [1]:
import torch
import torch.nn as nn

In [157]:
!wget https://raw.githubusercontent.com/hellocym/Transformer/main/utils.py

--2023-08-03 14:52:50--  https://raw.githubusercontent.com/hellocym/Transformer/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2145 (2.1K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   2.09K  --.-KB/s    in 0s      

2023-08-03 14:52:51 (15.7 MB/s) - ‘utils.py’ saved [2145/2145]



In [24]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2023-08-03 14:05:46.254530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-08-03 14:06:01.147770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 17.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [26]:
!mkdir -p .data/multi30k

In [29]:
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz

--2023-08-03 14:07:48--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1207136 (1.2M) [application/octet-stream]
Saving to: ‘training.tar.gz’

training.tar.gz     100%[===================>]   1.15M  --.-KB/s    in 0.007s  

2023-08-03 14:07:48 (169 MB/s) - ‘training.tar.gz’ saved [1207136/1207136]

--2023-08-03 14:07:48--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, a

In [30]:
!tar -xzf training.tar.gz -C .data/multi30k
!tar -xzf validation.tar.gz -C .data/multi30k
!tar -xzf mmt16_task1_test.tar.gz -C .data/multi30k

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'


In [31]:
!mv /content/.data/multi30k/test.de /content/.data/multi30k/test2016.de
!mv /content/.data/multi30k/test.en /content/.data/multi30k/test2016.en

In [32]:
!pip install -U 'torchtext==0.6.0'

# Structure

In [58]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert  (self.head_dim * heads == embed_size), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0] # Number of training examples
        values_len, keys_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into self.heads pieces
        values = values.reshape(N, values_len, self.heads, self.head_dim)
        keys = keys.reshape(N, keys_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values) # (N, values_len, heads, heads_dim)
        keys = self.keys(keys) # (N, keys_len, heads, heads_dim)
        query = self.queries(query) # (N, query_len, heads, heads_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # query shape: (N, query_len, heads, heads_dim)
        # keys shape: (N, keys_len, heads, heads_dim)
        # energy shape: (N, heads, query_len, keys_len)

        if mask is not None:
            energy = energy.masked_fill(mask==0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads*self.head_dim
        )
        # attention shape: (N, heads, query_len, keys_len)
        # values shape: (N, values_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then flatten last two dimensions

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape
        return out

In [59]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [60]:
class Encoder(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList([
            TransformerBlock(
                embed_size,
                heads,
                dropout=dropout,
                forward_expansion=forward_expansion,
            )
            for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = x.to(torch.long)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [61]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [62]:
class Decoder(nn.Module):
    def __init__(
            self,
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList([
            DecoderBlock(
                embed_size,
                heads,
                forward_expansion,
                dropout,
                device
            )
            for _ in range(num_layers)
        ])

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

In [238]:
class MyTransformer(nn.Module):
    def __init__(
            self,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,
            trg_pad_idx,
            embed_size=256,
            num_encoder_layers=6,
            num_decoder_layers=6,
            forward_expansion=4,
            heads=8,
            dropout=0,
            device="cuda",
            max_length=100,
    ):
        super(MyTransformer, self).__init__()
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_encoder_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_decoder_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, trg, src_mask=None, trg_mask=None):
        if True:
          src_mask = self.make_src_mask(src)
          trg_mask = self.make_trg_mask(trg)
        # print(src.shape)
        # print(trg.shape)
        enc_src = self.encoder(src, src_mask)
        # print(enc_src.shape)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        # print(out.shape)
        return out

In [239]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
print(device)
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2],[1, 5, 6, 4, 3, 9, 5, 2, 0]]).to(device)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2],[1, 7, 4, 3, 5, 9, 2, 0]]).to(device)

src_pad_idx1 = 0
trg_pad_idx1 = 0
src_vocab_size1 = 10
trg_vocab_size1 = 10
model1 = MyTransformer(src_vocab_size1, trg_vocab_size1, src_pad_idx1, trg_pad_idx1, device=device).to(
    device
)
out = model1(x, trg[:, :-1])

cpu


In [240]:
out.shape

torch.Size([3, 7, 10])

In [241]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = MyTransformer(
            embed_size=embedding_size,
            src_vocab_size=src_vocab_size,
            trg_vocab_size=trg_vocab_size,
            src_pad_idx=src_pad_idx,
            trg_pad_idx=src_pad_idx,
            heads=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            forward_expansion=forward_expansion,
            dropout=dropout,
            max_length=max_len,
            device=device,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        # embed_src = self.dropout(
        #     (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        # )
        # embed_trg = self.dropout(
        #     (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        # )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        assert len(src.shape) == 2, src.shape
        assert len(trg.shape) == 2, trg.shape
        assert src.shape[0] == trg.shape[0], (src.shape[0], trg.shape[0])
        out = self.transformer(
            src,
            trg,
            src_mask=src_padding_mask,
            trg_mask=trg_mask,
        )
        # print(out.shape)
        return out

# Training

In [242]:
import torch.optim as optim
import spacy
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [243]:
!pip install 'portalocker>=2.0.0'

In [244]:
spacy_ger = spacy.load('de_core_news_sm')
spacy_eng = spacy.load('en_core_web_sm')

In [245]:
def tokenize_ger(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

In [246]:
def tokenize_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [247]:
german = Field(tokenize=tokenize_ger, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_eng, lower=True, init_token='<sos>', eos_token='<eos>')

In [248]:
train_data, valid_data, test_data = Multi30k.splits(
    root='.data',
    exts=('.de', '.en'),
    fields=(german, english)
)

In [249]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [250]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
load_model = False
save_model = True

In [251]:
num_epochs = 5
learning_rate = 3e-4
batch_size = 32

In [252]:
src_vocab_size = len(german.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi['<pad>']

In [253]:
src_vocab_size

7853

In [254]:
writer = SummaryWriter('runs/loss_plot')
step = 0

In [255]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key = lambda x: len(x.src),
    device=device,
)

In [256]:
model = Transformer(
    embedding_size=embedding_size,
    src_vocab_size=src_vocab_size,
    trg_vocab_size=trg_vocab_size,
    src_pad_idx=src_pad_idx,
    num_heads=num_heads,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    forward_expansion=forward_expansion,
    dropout=dropout,
    max_len=max_len,
    device=device,
).to(device)

In [257]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [258]:
pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [320]:
import imp
import utils
utils = imp.reload(utils)
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint

In [260]:
if load_model:
  load_checkpoint(torch.load('my_checkpoint.pth.tar'), model, optimizer)

In [261]:
sentence = 'ein pferd geht unter einer brücke neben einem boot.'

In [263]:
for epoch in range(num_epochs):
  print(f'[Epoch {epoch} / {num_epochs}]')

  if save_model:
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }

    save_checkpoint(checkpoint)

  model.eval()
  translated_sentence = translate_sentence(
      model, sentence, german, english, device, max_length=100
  )
  print(f'Translatesd example sentence \n {translated_sentence}')
  model.train()

  for batch_idx, batch in enumerate(train_iterator):
    inp_data = batch.src.to(device)
    target = batch.trg.to(device)

    # print(inp_data.shape)
    # print(target.shape)

    output = model(inp_data.T, target[:-1].T)

    output = output.reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)
    optimizer.zero_grad()

    loss = criterion(output, target)
    loss.backward()

    torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)

    optimizer.step()

    writer.add_scalar('Training loss', loss, global_step=step)
    step += 1

[Epoch 0 / 5]
=> Saving checkpoint


<ipython-input-263-e7dbf3ea115e>:35: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)


[Epoch 1 / 5]
=> Saving checkpoint
[Epoch 2 / 5]
=> Saving checkpoint
[Epoch 3 / 5]
=> Saving checkpoint
[Epoch 4 / 5]
=> Saving checkpoint


In [293]:
output = model(inp_data.T, target[:-1].T)

In [299]:
output.argmax(2).shape

torch.Size([32, 18])

In [302]:
output.shape

torch.Size([32, 18, 5893])

In [295]:
output.reshape(-1, output.shape[2]).shape

torch.Size([576, 5893])

In [321]:
score = bleu(test_data, model, german, english, device)

In [322]:
print(f'Bleu score {score*100:.2f}')

Bleu score 0.00


In [323]:
translated_sentence = translate_sentence(
      model, sentence, german, english, device, max_length=100
  )

In [324]:
print(f'Translatesd example sentence \n {translated_sentence}')

Translatesd example sentence 
 ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']
